# Player Search
This notebook explores the use of these two endpoints in retrieving individual NBA player statstics:
- https://v2.nba.api-sports.io/players
- https://v2.nba.api-sports.io/players/statistics

The code from this notebook will be used in V1 of the _Player Search_ feature.

In [1]:
from rel_bball_analytics import app

import pandas as pd
import requests

In [2]:
base_url = app.config["SPORTS_API_URL"]
headers = {"x-rapidapi-key": app.config["SPORTS_API_KEY"]}

### Get Player Info from Name

In [3]:
PLAYER_COLUMNS = [
    "id",
    "firstname",
    "lastname",
    "birth_date",
    "country",
    "height_feet",
    "height_inches",
    "weight_pounds",
    "jersey",
    "is_active",
    "start_year",
    "pro_years",
    "college",
]

In [4]:
def get_raw_player_data(name):
    url = f"{base_url}players"
    params = {"name": name}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [5]:
def filter_nba_players(players):
    players["is_nba"] = players["leagues"].apply(lambda obj: "standard" in obj.keys())
    return players.loc[players.is_nba]

In [6]:
def clean_player_data(data):
    data["birth_date"] = data["birth"].apply(lambda obj: obj.get("date"))
    data["country"] = data["birth"].apply(lambda obj: obj.get("country"))
    data["start_year"] = data["nba"].apply(lambda obj: obj.get("start"))
    data["pro_years"] = data["nba"].apply(lambda obj: obj.get("pro"))
    data["height_feet"] = data["height"].apply(lambda obj: obj.get("feets"))
    data["height_inches"] = data["height"].apply(lambda obj: obj.get("inches"))
    data["weight_pounds"] = data["weight"].apply(lambda obj: obj.get("pounds"))
    data["jersey"] = data["leagues"].apply(lambda obj: obj["standard"].get("jersey"))
    data["is_active"] = data["leagues"].apply(lambda obj: obj["standard"].get("active"))
    data["position"] = data["leagues"].apply(lambda obj: obj["standard"].get("pos"))

    return data[PLAYER_COLUMNS]

In [7]:
player_data = get_raw_player_data(name="curry")
if player_data["results"] == 0:
    print("No player matches")

df_players = pd.DataFrame(player_data["response"])
df_players = filter_nba_players(players=df_players)
df_players = clean_player_data(data=df_players)

In [8]:
df_players

,id,firstname,lastname,birth_date,country,height_feet,height_inches,weight_pounds,jersey,is_active,start_year,pro_years,college
0,123,Seth,Curry,1990-08-23,USA,6,2,185,30,True,2013,7,Duke
1,124,Stephen,Curry,1988-03-14,USA,6,2,185,30,True,2009,12,Davidson


### Get Player Stats from ID

In [9]:
# Sample arguments to pass into main function
player_id = 124
season = 2021

In [10]:
EXCLUDED_STATS_COLUMNS = [
    "player",
    "team",
    "game",
    "comment",
]

In [11]:
def get_raw_stats_data(player_id, season=2022):
    url = f"{base_url}players/statistics"
    params = {"id": player_id, "season": season}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [12]:
def time_to_int(time):
    if time is None or type(time) == int:
        return time
    
    if ":" in time:
        mins, secs = time.split(":")
        result = int(mins) + int(secs) / 60
        return round(result, 2)

    return round(int(time), 2)

In [13]:
def clean_stats_data(data):
    data["player_id"] = data["player"].apply(lambda obj: obj["id"])
    data["team_id"] = data["team"].apply(lambda obj: obj["id"])
    data["team_name"] = data["team"].apply(lambda obj: obj["name"])
    data["team_code"] = data["team"].apply(lambda obj: obj["code"])
    data["game_id"] = data["game"].apply(lambda obj: obj["id"])
    data["min"] = data["min"].apply(time_to_int)

    return data.drop(EXCLUDED_STATS_COLUMNS, axis=1)

In [14]:
stats_data = get_raw_stats_data(player_id=player_id, season=season)
if stats_data["results"] == 0:
    print(f"No stats available")

df_stats = pd.DataFrame(stats_data["response"])
df_stats = df_stats.dropna(subset=["points", "min"])
df_stats = clean_stats_data(data=df_stats)

### Get Summary Stats (Per Game)

In [16]:
STATS_COLUMNS = {
    "points": "points",
    "min": "minutes_played",
    "fgm": "field_goals",
    "fga": "field_goal_attempts",
    "fgp": "field_goal_percentage",
    "tpm": "three_points_made",
    "tpa": "three_point_attempts",
    "tpp": "three_point_percentage",
    "ftm": "free_throws",
    "fta": "free_throw_attempts",
    "ftp": "free_throw_percentage",
    "offReb": "offensive_rebounds",
    "defReb": "defensive_rebounds",
    "totReb": "total_rebounds",
    "assists": "assists",
    "steals": "steals",
    "blocks": "blocks",
    "turnovers": "turnovers",
    "pFouls": "personal_fouls",
    "plusMinus": "plus_minus",
}

In [17]:
def get_summary_stats(data):
    stats_data = data[STATS_COLUMNS.keys()].rename(columns=STATS_COLUMNS)
    return stats_data.mean(axis=0, numeric_only=True) 

In [18]:
summary_stats = {
    "id": player_id,
    "season": season,
    "team_id": df_stats["team_id"].mode()[0],
    "team": df_stats["team_code"].mode()[0],
    "position": df_stats["pos"].mode()[0],
    "games_played": len(df_stats),
    **get_summary_stats(data=df_stats)
}

In [19]:
summary_stats

{'id': 124,
 'season': 2021,
 'team_id': 11,
 'team': 'GSW',
 'position': 'PG',
 'games_played': 92,
 'points': 25.869565217391305,
 'minutes_played': 33.71423913043478,
 'field_goals': 8.565217391304348,
 'field_goal_attempts': 19.141304347826086,
 'three_points_made': 4.445652173913044,
 'three_point_attempts': 11.391304347826088,
 'free_throws': 4.293478260869565,
 'free_throw_attempts': 4.815217391304348,
 'offensive_rebounds': 0.5,
 'defensive_rebounds': 4.641304347826087,
 'total_rebounds': 5.141304347826087,
 'assists': 5.923913043478261,
 'steals': 1.358695652173913,
 'blocks': 0.358695652173913,
 'turnovers': 3.0434782608695654,
 'personal_fouls': 2.097826086956522}